In [1]:
import numpy as np 
import pandas as pd 
import requests
from pprint import pprint # Импортируем функцию pprint()

from geopy.geocoders import Nominatim
nominaltim = Nominatim(user_agent='d3enjan')

[Cсылка на дата сет на всякий случай](https://disk.yandex.ru/d/TEXJ7_FiGXlS6w)

In [2]:
# Подгрузим наши данные из соревнования

DATA_DIR = 'C:/Users/d3en/OneDrive/Рабочий стол/dataScience/IDE/PROJECT-3. EDA + Feature Engineering. Соревнование на Kaggle/data_3/kaggle'
df_train = pd.read_csv(DATA_DIR+'/hotels_train.csv') # датасет для обучения
df_test = pd.read_csv(DATA_DIR+'/hotels_test.csv') # датасет для предсказания
sample_submission = pd.read_csv(DATA_DIR+'/submission.csv') # самбмишн

In [3]:
hotels = df_test.append(df_train, sort=False).reset_index(drop=True)
hotels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               515738 non-null  object 
 1   additional_number_of_scoring                515738 non-null  int64  
 2   review_date                                 515738 non-null  object 
 3   average_score                               515738 non-null  float64
 4   hotel_name                                  515738 non-null  object 
 5   reviewer_nationality                        515738 non-null  object 
 6   negative_review                             515738 non-null  object 
 7   review_total_negative_word_counts           515738 non-null  int64  
 8   total_number_of_reviews                     515738 non-null  int64  
 9   positive_review                             515738 non-null  object 
 

In [4]:
location_fr = nominaltim.geocode('France').raw
location_au = nominaltim.geocode('Austria').raw
location_It = nominaltim.geocode('Italy').raw
location_Am = nominaltim.geocode('Amsterdam').raw
location_Sp = nominaltim.geocode('Spain').raw
location_ld = nominaltim.geocode('London').raw

pprint(location_fr) # Это выдает запрос
pprint(location_fr['lat']) #Широта
pprint(location_fr['lon']) #Долгота

{'boundingbox': ['-50.2187169', '51.3055721', '-178.3873749', '172.3057152'],
 'class': 'boundary',
 'display_name': 'France',
 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons/poi_boundary_administrative.p.20.png',
 'importance': 1.0233264437396503,
 'lat': '46.603354',
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. '
            'https://osm.org/copyright',
 'lon': '1.8883335',
 'osm_id': 2202162,
 'osm_type': 'relation',
 'place_id': 298141959,
 'type': 'administrative'}
'46.603354'
'1.8883335'


In [5]:
# Функции заменяю значения в ячейках по условию

def lat_def(l_at):    
    l_at = l_at.split(' ')
    if 'France' in l_at:     
        return location_fr['lat']
    if  'Austria' in l_at:
        return location_au['lat']
    if 'Italy' in l_at:
        return location_It['lat']
    if 'Amsterdam' in l_at:
        return location_Am['lat']
    if 'Spain' in l_at:
        return location_Sp['lat']
    if 'London' in l_at:
        return location_ld['lat']

def lng_def(l_ng):  
    l_ng = l_ng.split(' ')
    if 'France' in l_ng:        
        return location_fr['lon']
    if  'Austria' in l_ng:
        return location_au['lon']
    if 'Italy' in l_ng:
        return location_It['lon']
    if 'Amsterdam' in l_ng:
        return location_Am['lon']
    if 'Spain' in l_ng:
        return location_Sp['lon']
    if 'London' in l_ng:
        return location_ld['lon'] 


In [6]:
hotels['lat'].isnull().value_counts()

False    512470
True       3268
Name: lat, dtype: int64

In [7]:
# Тут он заменяет значения в стобце lat и lng согласно функции, 
# тем самым меняет все значения. А хотелсоь бы поменять 3268 пустых значений, не трогая те которые есть
# Пропуски в обоих столбцах одинаковые
# Нужно поменять только страны Austria, France 
hotels['lat'] = hotels['hotel_address'].apply(lat_def)
hotels['lng'] = hotels['hotel_address'].apply(lng_def)

# И еще вопрос

# Как сделать так, чтобы он подставлял в ячейку сразу результат кода nominaltim.geocode('France').raw,
 
# где 'France', после split(' ') это то, что хранится в hotels['hotel_address'] = (Via Senigallia 6 20161 Milan Italy)- например, то есть и это значение ставил автоматически.

# Тогда там можно будет искать координаты по названию отеля, это более точно